In [1]:
%run Deck.ipynb
%run Player.ipynb
%run Board.ipynb

In [2]:
class MonteCarloTreeSearchNode():
    
    def __init__(self, state, parent=None, parent_action=None, name=None):
        self.state = state
        self.parent = parent
        self.name = name
        self.parent_action = parent_action
        self.children = []
        self._number_of_visits = 0
        self.t_score = 0

        self._results = defaultdict(int)
        self._results[1] = 0
        self._results[-1] = 0